<a href="https://colab.research.google.com/github/mantoan-thi/AutoML/blob/main/Day_Trading_with_Machine_Learning_(witn_H2O_AutoML_Automatic_Machine_Learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalações

In [1]:
!pip install h2o

     |████████████████████████████████| 175.8 MB 35 kB/s 
  Created wheel for h2o: filename=h2o-3.34.0.1-py2.py3-none-any.whl size=175823552 sha256=3d40dcfad75ba24ea9d826a905c78e05a813f508b964b2194bc8cffec89cfbc5
  Stored in directory: /root/.cache/pip/wheels/5c/7b/60/014a2b6d009793271276a9a41cb954659c8f23b8823ca21625
Successfully built h2o


In [2]:
!pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 5.4 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=cd22044c59912288fa96065ffefe66876a2991d9c19a30ca1725b4b49df4ef59
  Stored in directory: /tmp/pip-ephem-wheel-cache-ir0ibt72/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


# Importando as bibliotecas

In [3]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

# Treinando o modelo

In [4]:
def processar(dados, cls,tm):
  data = dados
  classe = cls
  tempo_maximo = tm
  h2o.init()
  # Identifica dinamicamente as colunas do arquivo csv
  colunas = data.columns.tolist()
  variaveis_independentes = [coluna for coluna in colunas if coluna != classe]

  # Divide os dados em treino e teste
  data = h2o.H2OFrame(data)
  treino, teste = data.split_frame(ratios=[.7])

  # Transforma a variavel dependente em fator
  treino[classe] = treino[classe].asfactor()
  teste[classe] = teste[classe].asfactor()

  # Auto ML
  # Busca o modelo valor gravado no atributo tempo_maximo em segundos. podemos em vez disso definir max_models
  modelo_automl = H2OAutoML(max_runtime_secs=tempo_maximo,sort_metric='AUC')
  #modelo_automl = H2OAutoML(max_models=20, seed=1)
  modelo_automl.train(y=classe, training_frame=treino)

  # Ranking dos melhores AutoML
  ranking = modelo_automl.leaderboard
  #ranking = ranking.as_data_frame()

  # Imprime todas as linhas em vez do padrão (10 linhas)
  ranking.head(rows=ranking.nrows)

  # O modelo líder é armazenado aqui
  modelo_automl.leader

  return ranking, modelo_automl,treino,teste

In [5]:
def prever(aml, dados_test):
  # Para gerar previsões em um conjunto de teste, você pode fazer previsões
  # diretamente no objeto `H2OAutoML` ou no modelo líder
  # objeto diretamente
  preds = aml.predict(dados_test)

  # ou:
  #preds = aml.leader.predict(dados_test)
  return preds

# Coletando e processando dados da Bolsa

importando as bibliotecas

In [6]:
import yfinance as yf #Instalando e importando a API no código

Coletando e processando

In [22]:
def coletar_dados(ativo,inicio,fim):
    df = yf.download(ativo, start=inicio, end=fim)
    df2 = df.copy()
    df.reset_index(inplace=True)
    df['Tendencia_3d'] = (df['Close'].shift(-1)-df['Open'].shift(-1))+(df['Close'].shift(-2)-df['Open'].shift(-2))+(df['Close'].shift(-3)-df['Open'].shift(-3))
    df['Low_open'] = df.Low-df.Open
    df['High_open'] = df.High-df.Open
    df['Variação'] = df['Adj Close'].pct_change()
    df.dropna(inplace=True)
    df.drop(columns=['Date'],axis=1, inplace=True)
    df['Dir'] = df.apply(lambda x: 1 if x['Tendencia_3d'] > 0 else 0, axis=1)
    return df,df2

Coletando dados do ITAU4

In [19]:
dados = coletar_dados('ITUB4.SA',"2009-01-01","2021-04-30")

[*********************100%***********************]  1 of 1 completed


In [10]:
dados.shape

(3041, 11)

In [11]:
# Colunas
dados.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Tendencia_3d',
       'Low_open', 'High_open', 'Variação', 'Dir'],
      dtype='object')

In [12]:
# Iremos pegar a coluna Dir
cls = 'Dir'
ranking, modelo_automl,treino,teste = processar(dados,cls,60)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqepq7trr
  JVM stdout: /tmp/tmpqepq7trr/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpqepq7trr/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.1
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_unknownUser_vct2fo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [13]:
# Ranking
ranking

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20210920_211817,1,0.00343861,1,0,0.0255555,0.000653086
XGBoost_2_AutoML_1_20210920_211817,1,0.00993303,1,0,0.01234,0.000152275
StackedEnsemble_BestOfFamily_2_AutoML_1_20210920_211817,1,0.00340326,1,0,0.0253579,0.000643021
XGBoost_1_AutoML_1_20210920_211817,1,0.0197673,1,0,0.0277952,0.000772576
StackedEnsemble_BestOfFamily_1_AutoML_1_20210920_211817,0.999987,0.00440107,0.999987,0.000959693,0.0300626,0.00090376
GBM_1_AutoML_1_20210920_211817,0.999985,0.00374587,0.999985,0.000959693,0.0306237,0.000937812
GLM_1_AutoML_1_20210920_211817,0.999959,0.0379737,0.99996,0.00335893,0.0958093,0.00917942
GBM_4_AutoML_1_20210920_211817,0.999905,0.0374697,0.999913,0.000959693,0.0483237,0.00233518
DRF_1_AutoML_1_20210920_211817,0.99979,0.0285302,0.999814,0.000959693,0.0536384,0.00287707
GBM_2_AutoML_1_20210920_211817,0.999714,0.0362193,0.999777,0.000959693,0.047342,0.00224126


In [14]:
# Test
teste

Open,High,Low,Close,Adj Close,Volume,Tendencia_3d,Low_open,High_open,Variação,Dir
11.901,12.1701,11.6361,11.9838,7.56228,8.66486e+06,0.0703707,-0.264928,0.269065,0.00242369,1
12.2156,12.3563,11.6982,11.7602,7.42123,7.5611e+06,-0.25665,-0.517435,0.140741,-0.0573985,0
11.7354,11.9465,11.5905,11.8306,7.46563,7.25116e+06,-0.633342,-0.144881,0.211113,0.0059828,0
11.8472,11.9838,11.4663,11.8182,7.4578,7.66377e+06,-1.33291,-0.380832,0.136602,-0.00104908,0
11.1849,11.396,10.7916,10.9034,6.88051,1.22583e+07,-1.20873,-0.39325,0.211114,-0.0365761,0
10.1417,10.4273,9.41731,10.2659,6.47823,1.99414e+07,-1.13836,-0.724408,0.285624,0.0180621,0
10.5184,10.5557,9.72777,9.91404,6.25619,1.64886e+07,-0.347716,-0.79064,0.0372553,-0.0342748,0
10.0589,10.1334,9.91404,10.0589,6.34761,5.36614e+06,-0.740967,-0.144881,0.0745106,0.0146137,0
9.90576,10.1955,9.37177,9.37177,5.91399,1.67628e+07,0.231809,-0.533992,0.289764,-0.0683127,1
9.5001,9.68637,9.27656,9.68637,6.11252,1.86427e+07,-0.0703716,-0.223533,0.186275,0.0335696,0


# Fazendo previsões

Testando com toda a base de dados

In [15]:
prever(modelo_automl,teste)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.000798264,0.999202
0,0.999212,0.000787567
0,0.999599,0.000400525
0,0.999888,0.00011154
0,0.999925,7.54579e-05
0,0.9999,9.99619e-05
0,0.999738,0.000262297
0,0.999818,0.000182486
1,0.000655242,0.999345
0,0.997864,0.00213636


Testando em apenas na primeira linha

In [16]:
prever(modelo_automl,teste[0:1,:])

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.000798264,0.999202


# Conhecendo o Modelo

In [17]:
modelo_automl.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_1_20210920_211817

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 3.626585831667453e-06
RMSE: 0.001904359690727425
LogLoss: 0.0005114664966812229
Null degrees of freedom: 2083
Residual degrees of freedom: 2079
Null deviance: 2889.0374485737066
Residual deviance: 2.1317923581673353
AIC: 12.131792358167335
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9524289421364434: 


,,0,1,Error,Rate
0,0,1042.0,0.0,0.0,(0.0/1042.0)
1,1,0.0,1042.0,0.0,(0.0/1042.0)
2,Total,1042.0,1042.0,0.0,(0.0/2084.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.952429,1.000000,181.0
1,max f2,0.952429,1.000000,181.0
2,max f0point5,0.952429,1.000000,181.0
3,max accuracy,0.952429,1.000000,181.0
4,max precision,0.999983,1.000000,0.0
5,max recall,0.952429,1.000000,181.0
6,max specificity,0.999983,1.000000,0.0
7,max absolute_mcc,0.952429,1.000000,181.0
8,max min_per_class_accuracy,0.952429,1.000000,181.0
9,max mean_per_class_accuracy,0.952429,1.000000,181.0



Gains/Lift Table: Avg response rate: 50.00 %, avg score: 50.01 %


...

,,0,1,Error,Rate
0,0,1042.0,0.0,0.0,(0.0/1042.0)
1,1,0.0,1042.0,0.0,(0.0/1042.0)
2,Total,1042.0,1042.0,0.0,(0.0/2084.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.206304,1.000000,179.0
1,max f2,0.206304,1.000000,179.0
2,max f0point5,0.206304,1.000000,179.0
3,max accuracy,0.206304,1.000000,179.0
4,max precision,0.999997,1.000000,0.0
5,max recall,0.206304,1.000000,179.0
6,max specificity,0.999997,1.000000,0.0
7,max absolute_mcc,0.206304,1.000000,179.0
8,max min_per_class_accuracy,0.206304,1.000000,179.0
9,max mean_per_class_accuracy,0.206304,1.000000,179.0



Gains/Lift Table: Avg response rate: 50.00 %, avg score: 50.00 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010077,0.999999,2.0,2.000000,1.0,0.999999,1.000000,0.999999,0.020154,0.020154,100.0,100.000000,0.020154
1,2,0.020154,0.999997,2.0,2.000000,1.0,0.999998,1.000000,0.999998,0.020154,0.040307,100.0,100.000000,0.040307
2,3,0.030230,0.999997,2.0,2.000000,1.0,0.999997,1.000000,0.999998,0.020154,0.060461,100.0,100.000000,0.060461
3,4,0.040307,0.999995,2.0,2.000000,1.0,0.999996,1.000000,0.999997,0.020154,0.080614,100.0,100.000000,0.080614
4,5,0.050384,0.999992,2.0,2.000000,1.0,0.999994,1.000000,0.999997,0.020154,0.100768,100.0,100.000000,0.100768
5,6,0.100288,0.999939,2.0,2.000000,1.0,0.999964,1.000000,0.999981,0.099808,0.200576,100.0,100.000000,0.200576
6,7,0.150192,0.999867,2.0,2.000000,1.0,0.999906,1.000000,0.999956,0.099808,0.300384,100.0,100.000000,0.300384
7,8,0.200096,0.999764,2.0,2.000000,1.0,0.999816,1.000000,0.999921,0.099808,0.400192,100.0,100.000000,0.400192
8,9,0.299904,0.999399,2.0,2.000000,1.0,0.999613,1.000000,0.999818,0.199616,0.599808,100.0,100.000000,0.599808
9,10,0.400192,0.998248,2.0,2.000000,1.0,0.998977,1.000000,0.999608,0.200576,0.800384,100.0,100.000000,0.800384


In [109]:
modelo_automl.key

'AutoML_1_20210920_211817@@Dir'

# Novas previsões

In [215]:
# Novos dados da bolsa
dados,dados2 = coletar_dados('ITUB4.SA',"2009-01-01","2021-09-21")

# transformando os dados para H20Frame
dados3 = h2o.H2OFrame(dados2)

# selecionando a ultima linha - 17/09
dados3[-2:-1,:]

# Salvando de H20Frame para Pandas
prev = prever(modelo_automl,teste[0-1:,:]).as_data_frame()

# Pegando a previsão
valor = int(prev.iloc[:,0].values)

[*********************100%***********************]  1 of 1 completed
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [216]:
# Verificando a linha -> 17/09
dados3[-2:-1,:]


Open,High,Low,Close,Adj Close,Volume
28.47,28.5,27.82,27.82,27.82,4.36199e+07


In [217]:
# Interpretando...
if valor ==1:
  print('Comprar')
else:
  print('Vender')

Vender


In [218]:
# Ultimos 5 dias.
dados2.tail(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-14,29.100000,29.360001,28.900000,28.990000,28.990000,24181000
2021-09-15,29.049999,29.070000,28.520000,28.520000,28.520000,26397500
2021-09-16,28.320000,28.660000,28.160000,28.500000,28.500000,46425000
2021-09-17,28.469999,28.500000,27.820000,27.820000,27.820000,43619900
2021-09-20,27.299999,27.629999,26.860001,27.190001,27.190001,45684200


# Checando as previsões

In [228]:
# Plotando.

# 1. Podemos ver que no dia 17/09 o preço era de 27.82.
# 2. s previsão para o dia 20/09 era venda.
# 3. Assim no dia 20/09 o preço caiu para 27.19, confirmando a previsão.
import plotly.graph_objects as go

fig = go.Figure([go.Scatter(x=dados2['Date'].tail(5), y=dados2['Adj Close'].tail(5))])
fig.show()